In [127]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [152]:
# run once
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

# 1. Reading in the Data

In [10]:
f = open("data\Alice in Wonderland.txt", "r", encoding="utf8")
data = f.read()
f.close()
print(data)

Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     Down the Rabbit-Hole
 CHAPTER II.    The Pool of Tears
 CHAPTER III.   A Caucus-Race and a Long Tale
 CHAPTER IV.    The Rabbit Sends in a Little Bill
 CHAPTER V.     Advice from a Caterpillar
 CHAPTER VI.    Pig and Pepper
 CHAPTER VII.   A Mad Tea-Party
 CHAPTER VIII.  The Queen’s Croquet-Ground
 CHAPTER IX.    The Mock Turtle’s Story
 CHAPTER X.     The Lobster Quadrille
 CHAPTER XI.    Who Stole the Tarts?
 CHAPTER XII.   Alice’s Evidence




CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very 

# 2. Text Preprocessing

### a. Expand Contractions
example: I'm -> I am

In [11]:
CONTRACTION_MAP = { 
"ain't": "am not",
"aren't": "am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [108]:
#test
print(expand_contractions('''test: it's I'll I'm haven't''',CONTRACTION_MAP))

test: it is I will I am have not


### b. Remove non-alphabetical characters
Remove special characters like periods, commas, as well as digits. Only keeping alphabet characters and spaces

In [118]:
def remove_non_alphabetical_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

In [119]:
#test
print(remove_non_alphabetical_characters('t !@#$%^&*()_+-={}[]:"e;''">? s ./\|<>,t1234567890`'))

t e s t


### c. Removing stopwords
example: remove 'a', 'the', 'and', etc.

In [39]:
def remove_stopwords(text):
    stopword_list = set(stopwords.words('english')) 
    
    tokens = word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    
    return ' '.join(filtered_tokens) 

In [128]:
#test
print(remove_stopwords('the test is a test to test the test'))

test test test test


### d. Stemming 
#### (may remove due to aggressive pruning issue + lemmatization more effective)
example: making -> make

In [130]:
def stemmer_text(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [168]:
print(stemmer_text('making make makes'))
print(stemmer_text('try tries ponies')) # issue with stemming - https://medium.com/@tusharsri/nlp-a-quick-guide-to-stemming-60f1ca5db49e

make make make
tri tri poni


### e. Lemmatization
example: tries -> try

In [165]:
def lemmatize_text(text):
    wn_lemmatizer = WordNetLemmatizer()
    text = ' '.join([wn_lemmatizer.lemmatize(word) for word in text.split()])
    return text

In [172]:
#test
print(lemmatize_text('making make makes'))
print(lemmatize_text('try tries ponies'))

making make make
try try pony


In [175]:
def preprocess(text):
    text = text.lower()
    text = expand_contractions(text)
    text = remove_non_alphabetical_characters(text)
    text = remove_stopwords(text)
    #text = stemmer_text(text)
    text = lemmatize_text(text)
    return text

In [182]:
preprocessed_text = preprocess(data)
#print(preprocessed_text)

# 3. Representation of words (bag of words for single document)
(If we have more documents, we can create sparse bag of words. Alice in Wonderland might not be enough data)

In [177]:
def bag_of_words(text):
    word_dict = {}
    for word in text:
        if word in word_dict.keys():
            word_dict[word] += 1
        else:
            word_dict[word] = 1
    return word_dict

In [190]:
dict_data = bag_of_words(word_tokenize(preprocessed_text))
print(dict_data)

{'alices': 13, 'adventure': 6, 'wonderland': 3, 'lewis': 1, 'carroll': 1, 'millennium': 1, 'fulcrum': 1, 'edition': 3, 'content': 2, 'chapter': 24, 'rabbithole': 4, 'ii': 3, 'pool': 11, 'tear': 12, 'iii': 2, 'caucusrace': 4, 'long': 33, 'tale': 5, 'iv': 2, 'rabbit': 48, 'sends': 2, 'little': 129, 'bill': 16, 'v': 2, 'advice': 3, 'caterpillar': 29, 'vi': 2, 'pig': 11, 'pepper': 8, 'vii': 2, 'mad': 15, 'teaparty': 3, 'viii': 2, 'queen': 77, 'croquetground': 4, 'ix': 2, 'mock': 57, 'turtle': 61, 'story': 9, 'x': 2, 'lobster': 15, 'quadrille': 5, 'xi': 2, 'stole': 3, 'tart': 8, 'xii': 2, 'evidence': 8, 'alice': 385, 'beginning': 13, 'get': 46, 'tired': 7, 'sitting': 10, 'sister': 10, 'bank': 3, 'nothing': 34, 'twice': 5, 'peeped': 3, 'book': 8, 'reading': 3, 'picture': 5, 'conversation': 11, 'use': 18, 'thought': 76, 'without': 25, 'considering': 3, 'mind': 10, 'well': 60, 'could': 77, 'hot': 5, 'day': 29, 'made': 30, 'feel': 8, 'sleepy': 5, 'stupid': 6, 'whether': 11, 'pleasure': 2, 'maki